In [ ]:
import altair as alt
import numpy as np
import pandas as pd
from vega_datasets import data

source = data.cars()

source.head()


base = (alt
 .Chart(source)
 .mark_point()
 .encode(x=alt.X('Miles_per_Gallon'),
        y=alt.Y('Acceleration'))
)

line=base.transform_regression('Miles_per_Gallon','Acceleration').mark_line()
loess=base.transform_loess('Miles_per_Gallon','Acceleration').mark_line(color='red')
poly=base.transform_regression('Miles_per_Gallon','Acceleration',method="poly").mark_line(color='green')

test=base.mark_text()
base+line+loess+poly

def get_decile(arr):
    perc=np.array([0,10,20,30,40,50,60,70,80,90,100])
    out=[round(np.percentile(arr,i),2) for i in perc]
    return out

df=(source
 .groupby(['Origin'])
 .agg(metrics=('Acceleration',get_decile))['metrics']
 .apply(pd.Series)
 .rename(columns=lambda x:str(x*10)+"%")
 .reset_index()
 .melt(id_vars='Origin')
)
df.head()

df_new=(source
 .assign(Miles_Per_Gallon=lambda x:np.round(x['Miles_per_Gallon'],0))
 .groupby(['Miles_per_Gallon'])
 .agg(metrics=('Acceleration',get_decile))['metrics']
 .apply(pd.Series)
 .rename(columns=lambda x:str(x*10)+"%")
 .reset_index()
 .melt(id_vars='Miles_per_Gallon')
)
df_new.head()

(alt
 .Chart(df_new)
 .encode(x=alt.X('Miles_per_Gallon')
        ,y=alt.Y('value')
        ,color=alt.Color('variable:N'))
 .mark_line()
).properties(width=500)

(alt
 .Chart(df)
 .encode(x=alt.X('Origin')
        ,y=alt.Y('value')
        ,color=alt.Color('variable:N'))
 .mark_line()
).properties(width=500)





# def get_decile(arr):
#     perc=np.array([0,10,20,30,40,50,60,70,80,90,100])
#     out=[round(np.percentile(arr,i),2) for i in perc]
#     #a=np.percentile(arr,100)
#     return out
# from plotnine import *
# from mizani.formatters import percent_format,custom_format

# from plotnine import *
# from mizani.formatters import percent_format,custom_format

# def percentile_plot(df_data,metrics,nice_name):
#     percentiles=df_data \
#     .dropna(subset=[metrics])\
#     .groupby(["variant"]).agg({metrics:get_decile})[metrics].apply(pd.Series)\
#     .rename(columns= lambda x: str(x*10)+"%" ).reset_index()\
#     .drop(["0%","100%"],1)\
#     .merge(df_data.dropna(subset=[metrics]).groupby(["variant"]).agg({metrics:np.mean}).reset_index()
#            ,on='variant')\
#     .rename(columns={metrics:"average"})

#     new_percentile=percentiles.set_index("variant")\
#     .unstack().reset_index()\
#     .rename(columns={"level_0":'Percentile',0:nice_name})
    
#     #print(percentiles.head())

#     dollar_formatter = custom_format('${:.2f}')

#     return (ggplot(data=new_percentile)
#     + aes(x='variant',y=nice_name,color='Percentile', group='Percentile')
#     + geom_point() 
#     + geom_line()
#     + xlab("variant")
#     + geom_hline(yintercept=0,linetype='--',color='r')
#     + ggtitle("Transactional {} distribution by offer type".format(nice_name))
#     + scale_y_continuous(labels=dollar_formatter)
#     )

# #percentile_plot(df_data,metrics='ogp_usd',nice_name='oGP$')

